<a href="https://colab.research.google.com/github/sreeman-11021996/Fashion-Recommendation-system/blob/main/train_resnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
!pip install import_ipynb
import import_ipynb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.6 MB 5.0 MB/s 


In [1]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [2]:
!kaggle datasets download -d paramaggarwal/fashion-product-images-small

 99% 557M/565M [00:03<00:00, 201MB/s]
100% 565M/565M [00:03<00:00, 185MB/s]


In [3]:
import zipfile
zip_ref = zipfile.ZipFile("/content/fashion-product-images-small.zip", "r")
zip_ref.extractall("/content")
zip_ref.close()

In [4]:
import tensorflow
from keras.models import Model

import numpy as np
import pandas as pd
import os
from tqdm import tqdm
import pickle
import matplotlib.pyplot as plt

In [83]:
from utils import *
from dataset import train_val_split,FashionDataset

In [84]:
#Loading the labels and removing missing images:
data=pd.read_csv('/content/styles.csv',usecols=["id","gender","masterCategory",
                                                 "subCategory","articleType",
                                                 "baseColour","season","usage"])

In [85]:
# training and validation dataset
dataset = FashionDataset(data)

[INFO]: Checking if all images are present


44446it [00:03, 14061.64it/s]

[INFO]: Dropping indices: [6697, 16207, 32324, 36399, 40022]


In [86]:
dataset.df.head()

,id,gender,masterCategory,subCategory,articleType,baseColour,season,usage
0,15970,Men,Apparel,Topwear,Shirts,Navy Blue,Fall,Casual
1,39386,Men,Apparel,Bottomwear,Jeans,Blue,Summer,Casual
2,59263,Women,Accessories,Watches,Watches,Silver,Winter,Casual
3,21379,Men,Apparel,Bottomwear,Track Pants,Black,Fall,Casual
4,53759,Men,Apparel,Topwear,Tshirts,Grey,Summer,Casual


In [56]:
train_data, val_data = train_val_split(data)
print(f"[INFO]: Number of training sampels: {len(train_data)}")
print(f"[INFO]: Number of validation sampels: {len(val_data)}")

[INFO]: Checking if all images are present


44446it [00:03, 13685.39it/s]

[INFO]: Dropping indices: [6697, 16207, 32324, 36399, 40022]
[INFO]: Number of training sampels: 39996
[INFO]: Number of validation sampels: 4444


In [ ]:
"""# training and validation data loader
train_dataloader = DataLoader(
    train_dataset, batch_size=batch_size, shuffle=True
)
val_dataloader = DataLoader(
    val_dataset, batch_size=batch_size, shuffle=False
)"""